In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import os

# Get the current working directory
base_dir = os.getcwd()

data_dir = 'C:/Users/HP/Desktop/Uni work/Projets/Prediction du Churn/data'

df_bigml_20 = pd.read_csv(os.path.join(data_dir, 'churn-bigml-20.csv'))
df_bigml_80 = pd.read_csv(os.path.join(data_dir, 'churn-bigml-80.csv'))
df = pd.concat([df_bigml_20, df_bigml_80], axis=0)

# Check the shape
print("Combined dataset shape:", df.shape)
df.head()

Combined dataset shape: (3333, 20)


,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,LA,117,408,No,No,0,184.5,97,31.37,351.6,80,29.89,215.8,90,9.71,8.7,4,2.35,1,False
1,IN,65,415,No,No,0,129.1,137,21.95,228.5,83,19.42,208.8,111,9.40,12.7,6,3.43,4,True
2,NY,161,415,No,No,0,332.9,67,56.59,317.8,97,27.01,160.6,128,7.23,5.4,9,1.46,4,True
3,SC,111,415,No,No,0,110.4,103,18.77,137.3,102,11.67,189.6,105,8.53,7.7,6,2.08,2,False
4,HI,49,510,No,No,0,119.3,117,20.28,215.1,109,18.28,178.7,90,8.04,11.1,1,3.00,1,False


In [3]:
# Convert target variable to binary
le = LabelEncoder()
df['Churn'] = le.fit_transform(df['Churn'])
print("Churn variable encoded (False=0, True=1):")
print(df['Churn'].value_counts())

Churn variable encoded (False=0, True=1):
Churn
0    2850
1     483
Name: count, dtype: int64


In [4]:
# Handle categorical variables
# First, identify categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns
print("Categorical columns:", list(categorical_cols))

# Convert 'yes'/'no' features to binary (1/0)
for col in ['International plan', 'Voice mail plan']:
    df[col] = df[col].map({'Yes': 1, 'No': 0})

Categorical columns: ['State', 'International plan', 'Voice mail plan']


In [5]:
# One-hot encode remaining categorical features
df = pd.get_dummies(df, columns=['State'], drop_first=True)

# Check the updated dataframe
print("DataFrame shape after encoding:", df.shape)
df.head()

DataFrame shape after encoding: (3333, 69)


,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,...,State_SD,State_TN,State_TX,State_UT,State_VA,State_VT,State_WA,State_WI,State_WV,State_WY
0,117,408,0,0,0,184.5,97,31.37,351.6,80,...,False,False,False,False,False,False,False,False,False,False
1,65,415,0,0,0,129.1,137,21.95,228.5,83,...,False,False,False,False,False,False,False,False,False,False
2,161,415,0,0,0,332.9,67,56.59,317.8,97,...,False,False,False,False,False,False,False,False,False,False
3,111,415,0,0,0,110.4,103,18.77,137.3,102,...,False,False,False,False,False,False,False,False,False,False
4,49,510,0,0,0,119.3,117,20.28,215.1,109,...,False,False,False,False,False,False,False,False,False,False


In [9]:
# Feature engineering
# Create new features that might be useful
df['Minutes_per_call'] = df['Total day minutes'] / df['Total day calls']
df['Charge_per_minute'] = df['Total day charge'] / df['Total day minutes']


# Fill NaN values created by division by zero
df.fillna(0, inplace=True)

# Check the new features
df[['Minutes_per_call', 'Charge_per_minute']].describe()

,Minutes_per_call,Charge_per_minute
count,3333.000000,3333.000000
mean,1.871421,0.169901
std,0.732772,0.004164
min,0.000000,0.000000
25%,1.379592,0.169989
50%,1.790517,0.170004
75%,2.261765,0.170017
max,7.223333,0.170513


In [10]:
# Remove redundant features
# Since charges are directly calculated from minutes, we can remove them
cols_to_drop = ['Total day charge', 'Total eve charge', 'Total night charge', 'Total intl charge']
df = df.drop(columns=cols_to_drop)

print("Dataframe shape after dropping redundant features:", df.shape)

Dataframe shape after dropping redundant features: (3333, 67)


In [11]:
# Standardize numerical features
numerical_cols = [col for col in df.select_dtypes(include=['int64', 'float64']).columns 
                  if col != 'Churn']

scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Check the scaled data
df[numerical_cols].describe()

,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total eve minutes,Total eve calls,Total night minutes,Total night calls,Total intl minutes,Total intl calls,Customer service calls,Minutes_per_call,Charge_per_minute
count,3.333000e+03,3.333000e+03,3.333000e+03,3.333000e+03,3.333000e+03,3.333000e+03,3.333000e+03,3.333000e+03,3.333000e+03,3.333000e+03,3.333000e+03,3.333000e+03,3.333000e+03,3.333000e+03,3.333000e+03,3.333000e+03
mean,1.449652e-16,4.263683e-16,-2.558210e-17,-6.928485e-17,7.674629e-17,-3.155125e-16,-1.982612e-16,-7.568037e-17,3.309684e-16,8.100997e-17,-5.143067e-17,-3.400287e-16,-6.395524e-18,3.197762e-18,1.684155e-16,6.425370e-15
std,1.000150e+00,1.000150e+00,1.000150e+00,1.000150e+00,1.000150e+00,1.000150e+00,1.000150e+00,1.000150e+00,1.000150e+00,1.000150e+00,1.000150e+00,1.000150e+00,1.000150e+00,1.000150e+00,1.000150e+00,1.000150e+00
min,-2.513172e+00,-6.888343e-01,-3.275805e-01,-6.183963e-01,-5.917599e-01,-3.301096e+00,-5.005247e+00,-3.963622e+00,-5.025911e+00,-3.513648e+00,-3.429870e+00,-3.667413e+00,-1.820289e+00,-1.188218e+00,-2.554275e+00,-4.080963e+01
25%,-6.797448e-01,-6.888343e-01,-3.275805e-01,-6.183963e-01,-5.917599e-01,-6.624241e-01,-6.695701e-01,-6.780300e-01,-6.583610e-01,-6.698545e-01,-6.699340e-01,-6.223690e-01,-6.011951e-01,-4.279320e-01,-6.712900e-01,2.109521e-02
50%,-1.627644e-03,-5.236033e-01,-3.275805e-01,-6.183963e-01,-5.917599e-01,-6.887677e-03,2.812491e-02,8.276141e-03,-5.738630e-03,6.485803e-03,-5.505089e-03,2.246393e-02,-1.948306e-01,-4.279320e-01,-1.104239e-01,2.471983e-02
75%,6.513740e-01,1.718817e+00,-3.275805e-01,1.617086e+00,8.695542e-01,6.725198e-01,6.759846e-01,6.768330e-01,6.970854e-01,6.808485e-01,6.589239e-01,6.672969e-01,6.178983e-01,3.323545e-01,5.327749e-01,2.774944e-02
max,3.564766e+00,1.718817e+00,3.052685e+00,1.617086e+00,3.134591e+00,3.140422e+00,3.217588e+00,3.209066e+00,3.508382e+00,3.839081e+00,3.827739e+00,3.497397e+00,6.307001e+00,5.654360e+00,7.304748e+00,1.469036e-01


In [12]:
# Split the data into features and target
X = df.drop('Churn', axis=1)
y = df['Churn']

# Save the cleaned data
df.to_csv('cleaned_telecom_data.csv', index=False)

print("Data cleaning completed. Dataset saved as 'cleaned_telecom_data.csv'")
print("Final dataset shape:", df.shape)

Data cleaning completed. Dataset saved as 'cleaned_telecom_data.csv'
Final dataset shape: (3333, 67)
